In [1]:
import os 
import pandas as pd 
import json
from tqdm import tqdm
import collections
from itertools import chain 
import statsmodels
from statsmodels.stats import inter_rater as irr
import numpy as np 

In [25]:
prod_versions = ["v8", "v9", "v12", "v13"]
prod_versions = ["v14", "v15"]
prod_versions = ["v17"]
annotation_path = "../annotations/"

In [26]:
def get_qa_format(qas):
    return [{
        "qa_id": qa["questionId"],
        "sent_id": qa["sentId"],
        "predicate_idx": int(qa["predicateId"].split("-")[0]),
        "predicate_pos": qa["predicatePos"],
        "predicate": qa["predicate"],
        "question": qa["question"],
        "answer_idx": qa["answerId"],
        "answer":  qa["answer"]
    } for qa in qas]

def get_span_format(spans):
    return [{
        "id": span["id"],
        "start": span["start"],
        "end": span["end"],
        "qaIds": span["qaIds"]
    } for span in spans if not span["predicate"] and not span["include_predicate"]]

In [27]:
'''
structure of sample:

{
    "source": "",
    "summary": [[]] # list of sentences
    "qas": [
        "qa_id": 0
        "sent_id": 0
        "predicate_idx": "",
        "answer_idx": [],
        "question": "",
        "answer":  "",
        "model": ""
        "consistent": [True, True, True],
    ],
    "sourceId": 0,
    "token_labels": [[]], 
    "sentence_labels": [],
    "dataset": "cliff",
    "origin": "xsum",
    "model": "bart"
}
'''

'\nstructure of sample:\n\n{\n    "source": "",\n    "summary": [[]] # list of sentences\n    "qas": [\n        "qa_id": 0\n        "sent_id": 0\n        "predicate_idx": "",\n        "answer_idx": [],\n        "question": "",\n        "answer":  "",\n        "model": ""\n        "consistent": [True, True, True],\n    ],\n    "sourceId": 0,\n    "token_labels": [[]], \n    "sentence_labels": [],\n    "dataset": "cliff",\n    "origin": "xsum",\n    "model": "bart"\n}\n'

In [28]:
'''
go through the prod annotations 
build the sructure 
read the original file to add missing fields
'''

'\ngo through the prod annotations \nbuild the sructure \nread the original file to add missing fields\n'

# load cliff

In [29]:
def process_annotation(summary):
    df_spans = pd.DataFrame([x for x in summary["spans"] if "label" in x])
    df_spans["label"] = df_spans["label"].astype(int)
    df_wrong_spans = df_spans[df_spans["label"] == 0]
    wrong_qas = set(df_wrong_spans.explode("qaIds")["qaIds"].unique())

    df_qas = pd.DataFrame(summary["qas"])
    df_qas.loc[df_qas["questionId"].isin(wrong_qas), "label"] = 1
    df_qas["label"] = df_qas["label"].astype(int)
    return df_qas

In [30]:
released_data = []
for version in prod_versions:
    dir_path = os.path.join(annotation_path, version)
    for source_id in os.listdir(dir_path):
        if not os.path.isdir(os.path.join(dir_path, source_id)):
            continue

        # load input file to mturk 
        # origin_path = f"../src/data/cliff_multiple_summaries/{source_id}.json"
        # origin_path = f"../src/data/factscore_filtered/{source_id}.json"
        origin_path = f"../src/data/tofueval_chosen/mediasum/{source_id}.json"
        with open(origin_path, "r") as f:
            origin = json.load(f)

        annotations = []
        for annotator in os.listdir(os.path.join(dir_path, source_id)):
            annotator_path = os.path.join(dir_path, source_id, annotator)
            with open(annotator_path, "r") as f:
                annotator = json.load(f)
            annotations.append(annotator)

        if len(annotations) != 3:
            continue
        
        for i, summary in enumerate(origin["summaries"]):
            obj = {
                "source_id": origin["sourceId"],
                "source": [x["text"] for x in origin["source"]],
                "summary": [[x["text"] for x in summary["tokens"]]],
                # "spans": get_span_format(summary["spans"]),
                # "qas": get_qa_format(summary["qas"]),
                # "cliff_labels": [[x["label"] for x in summary["tokens"]]], 
                "model": summary["summaryId"],
                "datasource": origin["datasource"],
                "dataset": origin["dataset"],
            }

            obj["qas"] = get_qa_format(annotations[0]["summaries"][i]["qas"])
            obj["spans"] = get_span_format(annotations[0]["summaries"][i]["spans"])
            

            
            span_annotations = collections.defaultdict(list)
            for annot in annotations:
                for span in annot["summaries"][i]["spans"]:
                    span_id = span["id"]
                    if "label" in span:
                        span_label = span["label"]
                        span_annotations[span_id].append(span_label)

            for j, span in enumerate(obj["spans"]):
                obj["spans"][j]["annotations"] = span_annotations[span["id"]]

            qa_annotations = collections.defaultdict(list)
            for annot in annotations:
                df_qas = process_annotation(annot["summaries"][i])
                for row_id, qa in df_qas.iterrows():
                # for qa in annot["summaries"][i]["qas"]:
                    qa_id = qa["questionId"]
                    qa_label = qa["label"]  # 0 if "label" in qa and qa["label"] == 0 else 1
                    qa_annotations[qa_id].append(qa_label)

            for j, qa in enumerate(obj["qas"]):
                if len(qa_annotations[qa["qa_id"]]) != 3:
                    print(source_id, i, j, len(qa_annotations[qa["qa_id"]]))
                obj["qas"][j]["annotations"] = qa_annotations[qa["qa_id"]]

            released_data.append(obj)

CNN-388107_Shadow foreign policy 0 17 1
CNN-388107_Shadow foreign policy 0 18 1
CNN-388107_Shadow foreign policy 1 19 1
CNN-388107_Shadow foreign policy 1 20 1
CNN-340132_Trump administration's peace plan 0 22 1
CNN-340132_Trump administration's peace plan 0 23 1


KeyError: 'label'

In [10]:
df = pd.DataFrame(released_data)
df.head()

,source_id,source,summary,model,datasource,dataset,qas,spans
0,CNN-25553_Airlines' commitment to service impr...,"[CAROL, LIN, ,, CNN, ANCHOR, :, Well, ,, the, ...","[[The, airlines, promised, service, improvemen...",Model-Extra,mediasum,tofueval,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 2...","[{'id': 0, 'start': 1, 'end': 2, 'qaIds': [0, ..."
1,CNN-25553_Airlines' commitment to service impr...,"[CAROL, LIN, ,, CNN, ANCHOR, :, Well, ,, the, ...","[[Airlines, ', commitment, to, service, improv...",B,mediasum,tofueval,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 1...","[{'id': 0, 'start': 0, 'end': 1, 'qaIds': [4, ..."
2,CNN-164885_Cuban celebration and government ga...,"[FEYERICK, :, We, 'll, get, to, Donald, Trump,...","[[Cuba, celebrated, 50, years, since, defeatin...",Model-Extra,mediasum,tofueval,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 1...","[{'id': 0, 'start': 0, 'end': 1, 'qaIds': [0, ..."
3,CNN-164885_Cuban celebration and government ga...,"[FEYERICK, :, We, 'll, get, to, Donald, Trump,...","[[Cubans, celebrated, the, 50th, anniversary, ...",E,mediasum,tofueval,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 1...","[{'id': 0, 'start': 0, 'end': 1, 'qaIds': [0, ..."
4,CNN-7972_Federal Reserve's policy,"[DEBORAH, MARCHINI, ,, CNN, ANCHOR, :, Where, ...","[[Sandy, Weill, expects, another, Fed, rate, h...",Model-Extra,mediasum,tofueval,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 2...","[{'id': 0, 'start': 0, 'end': 2, 'qaIds': [0],..."


In [9]:
# output_path = "../annotations/prod/cliff.jsonl"
output_path = "../annotations/prod/factscore.jsonl"

In [11]:
df.shape

(24, 8)

In [11]:
df.to_json(output_path, orient="records", lines=True)

In [16]:
df["annotations"] = df["qas"].apply(lambda lst: [x["annotations"] for x in lst])
all_qas = [x for x in list(chain.from_iterable(df["annotations"])) if len(x) > 0]

In [24]:
sum(1 for x in all_qas if len(x) == 3)

546

In [20]:
np.array(all_qas)

/var/folders/c4/yrc5nyls2cn1fp3nhbcmvg040000gn/T/ipykernel_48845/2090474237.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(all_qas)


array([list([0, 0, 0]), list([0, 0, 0]), list([0, 0, 0]), list([0, 0, 0]),
       list([0, 0, 0]), list([0, 0, 0]), list([0, 1, 0]), list([1, 0, 1]),
       list([0, 0, 0]), list([0, 1, 0]), list([1, 0, 0]), list([0, 0, 0]),
       list([0, 0, 0]), list([0, 0, 1]), list([0, 0, 0]), list([0, 0, 0]),
       list([0, 0, 0]), list([0, 0, 1]), list([0, 1, 0]), list([0, 0, 0]),
       list([0, 1, 0]), list([0, 1, 1]), list([0, 0, 0]), list([0]),
       list([0]), list([1]), list([0]), list([0, 0, 0]), list([0, 0, 0]),
       list([0, 0, 0]), list([0, 0, 0]), list([0, 0, 0]), list([0, 0, 0]),
       list([0, 0, 0]), list([0, 0, 0]), list([1, 0, 0]), list([0, 0, 0]),
       list([0, 0, 0]), list([0, 0, 1]), list([0, 0, 0]), list([0, 0, 0]),
       list([0, 1, 0]), list([0, 0, 0]), list([0, 0, 0]), list([0, 0, 0]),
       list([0, 0, 0]), list([0, 0, 0]), list([0, 0, 0]), list([0, 0, 0]),
       list([0]), list([0]), list([1]), list([1]), list([0]), list([0]),
       list([0]), list([0, 0, 0]),

In [19]:
table = statsmodels.stats.inter_rater.aggregate_raters(np.array(all_qas))
fleiss = statsmodels.stats.inter_rater.fleiss_kappa(table[0], method='fleiss')
fleiss 

/var/folders/c4/yrc5nyls2cn1fp3nhbcmvg040000gn/T/ipykernel_48845/320864670.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  table = statsmodels.stats.inter_rater.aggregate_raters(np.array(all_qas))


ValueError: object of too small depth for desired array

In [15]:
table[0].sum()

2805

In [16]:
'''
structure of sample:

{
    "source": [],
    "summary": [[]] # list of sentences
    "qas": [
        "qa_id": 0
        "sent_id": 0
        "predicate_idx": "",
        "answer_idx": [],
        "question": "",
        "answer":  "",
        "model": ""
        "consistent": [True, True, True],
    ],
    "sourceId": 0,
    "token_labels": [[]], 
    "sentence_labels": [],
    "dataset": "cliff",
    "origin": "xsum",
    "model": "bart"
}
'''


'\nstructure of sample:\n\n{\n    "source": [],\n    "summary": [[]] # list of sentences\n    "qas": [\n        "qa_id": 0\n        "sent_id": 0\n        "predicate_idx": "",\n        "answer_idx": [],\n        "question": "",\n        "answer":  "",\n        "model": ""\n        "consistent": [True, True, True],\n    ],\n    "sourceId": 0,\n    "token_labels": [[]], \n    "sentence_labels": [],\n    "dataset": "cliff",\n    "origin": "xsum",\n    "model": "bart"\n}\n'

In [42]:
obj = {
    "source_id": origin["sourceId"],
    "source": origin["source"],
    "summary": [[x["text"] for x in origin["summaries"][0]["tokens"]]],
    "spans": get_span_format(origin["summaries"][0]["spans"]),
    "qas": get_qa_format(origin["summaries"][0]["qas"]),
    "cliff_labels": [[x["label"] for x in origin["summaries"][0]["tokens"]]], 
    "model": origin["summaries"][0]["summaryId"],
    "datasource": origin["datasource"],
    "dataset": origin["dataset"],
}

In [50]:
obj["spans"]

[{'id': 0,
  'start': 1,
  'end': 3,
  'qaIds': [8, 13, 17, 0, 4],
  'annotations': [1, 1, 1]},
 {'id': 1,
  'start': 1,
  'end': 6,
  'qaIds': [9, 14, 1, 5],
  'annotations': [1, 1, 1]},
 {'id': 2, 'start': 4, 'end': 5, 'qaIds': [18], 'annotations': [1, 1, 1]},
 {'id': 6, 'start': 10, 'end': 12, 'qaIds': [6], 'annotations': [1, 1, 1]},
 {'id': 10,
  'start': 14,
  'end': 15,
  'qaIds': [11, 19],
  'annotations': [1, 1, 1]},
 {'id': 12, 'start': 20, 'end': 22, 'qaIds': [12], 'annotations': [1, 1, 1]}]

In [ ]:
# add labels 


In [47]:
span_annotations = collections.defaultdict(list)
for annot in annotations:
    for span in annot["summaries"][0]["spans"]:
        span_id = span["id"]
        if "label" in span:
            span_label = span["label"]
            span_annotations[span_id].append(span_label)


In [52]:
qa_annotations = collections.defaultdict(list)
for annot in annotations:
    for qa in annot["summaries"][0]["qas"]:
        qa_id = qa["questionId"]
        qa_label = 1 if "label" in qa and qa["label"] == 0 else 0
        qa_annotations[qa_id].append(qa_label)

In [53]:
qa_annotations

defaultdict(list,
            {0: [1, 1, 1],
             1: [1, 1, 1],
             2: [1, 1, 1],
             3: [1, 0, 1],
             4: [1, 1, 1],
             5: [1, 1, 1],
             6: [1, 1, 1],
             7: [1, 1, 1],
             8: [1, 1, 1],
             9: [1, 1, 1],
             10: [1, 1, 1],
             11: [1, 1, 1],
             12: [1, 1, 1],
             13: [1, 1, 1],
             14: [1, 1, 1],
             15: [1, 1, 1],
             16: [1, 1, 1],
             17: [1, 1, 1],
             18: [1, 1, 1],
             19: [1, 1, 1]})

In [49]:
for i, span in enumerate(obj["spans"]):
    obj["spans"][i]["annotations"] = span_annotations[span["id"]]